In [42]:
import requests
import json
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 0. Consulta

In [2]:
registro_i = requests.get("https://www.boliviasegura.gob.bo/wp-content/json/api.php")

In [3]:
registro_i_json = registro_i.json()
# Elementos en fecha de entrada
# ['fecha', 'departamento', 'contador', 'total', 'porcentaje']

In [66]:
with open("20200423.json","r") as json_arch : 
    registro_i_json = json.load(json_arch)

In [67]:
registro_i_json["fecha"] # retroceder un día

'23/04/20 20:57'

In [68]:
#with open('{}.json'.format(pd.to_datetime(registro_i_json["fecha"]).strftime("%Y%m%d")), 'w') as outfile:
#    json.dump(registro_i_json, outfile)

## 1. Parseo del imput para cifras acumuladas

### Por departamentos

In [69]:
day_i = {}
for key, value in registro_i_json["departamento"].items()  :
    val_j =  value["contador"]
    val_j["total"]  = value["total"] 
    day_i[key] = val_j

df_day_i = pd.DataFrame(day_i).T.rename_axis(["depto"]).reset_index()
df_day_i["fecha"] = pd.to_datetime(registro_i_json["fecha"]).strftime("%Y-%m-%d")
df_day_i["depto"] = df_day_i["depto"].str.upper()
df_day_i["activos"] = df_day_i.confirmados - df_day_i.decesos - df_day_i.recuperados
df_day_i

,depto,confirmados,decesos,recuperados,sospechosos,descartados,total,fecha,activos
0,LP,150,13,9,4,1138,1314,2020-04-23,128
1,CB,66,6,13,0,859,944,2020-04-23,47
2,SC,385,19,14,0,1674,2092,2020-04-23,352
3,OR,62,2,8,0,251,323,2020-04-23,52
4,PT,21,1,0,20,103,145,2020-04-23,20
5,TJ,3,0,0,3,36,42,2020-04-23,3
6,CH,2,0,0,1,115,118,2020-04-23,2
7,BN,2,1,0,0,46,49,2020-04-23,1
8,PN,12,1,0,1,70,84,2020-04-23,11


### A nivel nacional

In [70]:
bol_i = registro_i_json["contador"]
bol_i["total"] = registro_i_json["total"]

sr_bol_i = pd.Series(bol_i)
sr_bol_i["fecha"] = pd.to_datetime(registro_i_json["fecha"]).strftime("%Y-%m-%d")
sr_bol_i["depto"] = "BOL"
sr_bol_i["activos"] = sr_bol_i.confirmados - sr_bol_i.decesos - sr_bol_i.recuperados
sr_bol_i


confirmados           703
decesos                43
recuperados            44
sospechosos            29
descartados          4292
total                5111
fecha          2020-04-23
depto                 BOL
activos               616
dtype: object

# 2. Calculando variaciones

In [71]:
conn = sqlite3.connect('BD_COVID19_BOL.sqlite')
cur = conn.cursor()

valores = [j.to_list() for i,j  in df_day_i.iterrows()]
cur.executemany("""INSERT INTO daily_covid19_BO(daily_depto
,daily_total_confirmados
,daily_total_decesos
,daily_total_recuperados
,daily_total_sospechosos
,daily_total_descartados
,daily_total_total
,daily_fecha
,daily_total_activos) VALUES (?,?,?,?,?,?,?,?,?);""", valores)
conn.commit()
conn.close()


### Extrayendo el acumulado del día anterior

In [72]:
# Extraer de base de datos los elememtos del día anterior para obtener las variaciones

In [73]:
fecha_actualizacion = pd.to_datetime(registro_i_json["fecha"]).strftime("%Y-%m-%d")
fecha_actualizacion

'2020-04-23'

In [114]:
fecha_actualizacion = '2020-04-21'

In [115]:
conn = sqlite3.connect('BD_COVID19_BOL.sqlite')

df_t_1 = pd.read_sql_query("""select * 
from daily_covid19_BO
where daily_fecha = strftime('%Y-%m-%d',date('{fecha_consulta}', '-1 day'))
order by daily_depto """.format(fecha_consulta = fecha_actualizacion), conn)

df_t = pd.read_sql_query("""select * 
from daily_covid19_BO
where daily_fecha = '{fecha_consulta}'
order by daily_depto """.format(fecha_consulta = fecha_actualizacion), conn)


conn.close()


delta = df_t[[  'daily_total_confirmados', 'daily_total_activos',
       'daily_total_decesos', 'daily_total_recuperados',
       'daily_total_sospechosos', 'daily_total_descartados',
       'daily_total_total']].values -   df_t_1[[  'daily_total_confirmados', 'daily_total_activos',
       'daily_total_decesos', 'daily_total_recuperados',
       'daily_total_sospechosos', 'daily_total_descartados',
       'daily_total_total']].values

delta_df = pd.DataFrame(delta , columns=['daily_delta_confirmados',
'daily_delta_activos', 'daily_delta_decesos', 'daily_delta_recuperados',
'daily_delta_sospechosos', 'daily_delta_descartados',
'daily_delta_total'])

delta_df[['daily_id', 'daily_fecha', 'daily_depto']] = df_t[['daily_id', 'daily_fecha', 'daily_depto']]

valores_delta = [j.to_list() for i,j  in delta_df.iterrows()]

conn = sqlite3.connect('BD_COVID19_BOL.sqlite')
cur = conn.cursor()

for val_i in valores_delta : 
    cur.execute("""UPDATE  daily_covid19_BO
    SET   daily_delta_confirmados = {0}
    , daily_delta_activos     =     {1} 
    , daily_delta_decesos     =     {2}
    , daily_delta_recuperados =  {3}
    , daily_delta_sospechosos =   {4}
    , daily_delta_descartados =   {5}
    , daily_delta_total       =   {6}                   
    where daily_id = {7}
    and daily_fecha = '{8}'
    and daily_depto = '{9}'  """.format(*val_i))
conn.commit()
conn.close()